#  SAR
## Load and Prepare Data
### Load packages

In [102]:
# load packages
import geopandas as gpd
import pandas as pd
import numpy as np
from pysal.lib import weights
from pysal.explore import esda
from spreg import ML_Lag

ImportError: cannot import name 'Impacts' from 'spreg.diagnostics_panel' (D:\anaconda3\Lib\site-packages\spreg\diagnostics_panel.py)

### Load Data

In [67]:
# load spatial data
gdf = gpd.read_file("E:\\study\\CASAterm1\\CASA0013_FSDS\\group_research\\SAR data\\greater_london.gpkg")

# load csv
tourism_df = pd.read_csv("E:\\study\\CASAterm1\\CASA0013_FSDS\\group_research\\SAR data\\londonT&NTlist.csv") 
turnover_df = pd.read_csv("E:\\study\\CASAterm1\\CASA0013_FSDS\\group_research\\SAR data\\turnover_final_merge.csv")
listings_df = pd.read_csv("E:\\study\\CASAterm1\\CASA0013_FSDS\\group_research\\SAR data\\listings.csv") 

view columns name

In [68]:
print(listings_df.columns)
print(gdf.columns)
print(tourism_df.columns)

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'license'],
      dtype='object')
Index(['geo_code', 'geo_label', 'geo_labelw', 'geometry'], dtype='object')
Index(['Unnamed: 0', 'MSOA_CODE', 'hotspot_binary'], dtype='object')


### Merge Data

In [69]:
# Rename gdf's geo code for merging data
gdf = gdf.rename(columns={'geo_code': 'MSOA_CODE'})
turnover_df = turnover_df.rename(columns={'geo_code':'MSOA_CODE'})

In [106]:
# 定义与旅游相关的行业关键词
tourism_categories = ['_acmd', '_f&b', '_travel', '_art&ent', '_sp_recrea', '_gambling']

# 从turnover_df的列名中筛选出含有上述任一行业关键词的列
tourism_cols = [col for col in turnover_df.columns if any(cat in col for cat in tourism_categories)]

# 将所有相关列相加形成一个综合指标
# axis=1表示按行求和
turnover_df['tourism_economic_indicator'] = turnover_df[tourism_cols].sum(axis=1)

In [71]:
listings_gdf = gpd.GeoDataFrame(
    listings_df,
    geometry=gpd.points_from_xy(listings_df['longitude'], listings_df['latitude']),
    crs="EPSG:4326"
)
listings_gdf = listings_gdf.to_crs(epsg=27700)

# 确保gdf的投影与listings_gdf一致，如有需要：
gdf = gdf.to_crs(epsg=27700)

# 使用空间连接将每个Airbnb房源分配到MSOA区域
listings_with_area = gpd.sjoin(listings_gdf, gdf, how="left",predicate="within")

# 按MSOA_CODE汇总Airbnb房源数
airbnb_supply = listings_with_area.groupby('MSOA_CODE').size().reset_index(name='airbnb_supply')

#-------------------------
# 合并数据
#-------------------------
# Select only the required column from tourism_df along with 'MSOA_CODE'
tourism_df_selected = turnover_df[['MSOA_CODE', 'tourism_economic_indicator']]

# 如果你只需要从 turnover_df 中获取部分列，直接使用选定列进行合并
merged_df = tourism_df_selected.copy()

# 将Airbnb供应信息合并
merged_df = pd.merge(merged_df, airbnb_supply, on='MSOA_CODE', how='left')
merged_df['airbnb_supply'] = merged_df['airbnb_supply'].fillna(0)  # 没有房源的区域记为0

# 将 tourism_df 数据合并
merged_df = pd.merge(merged_df, tourism_df, on='MSOA_CODE', how='left')

# 将数据与 gdf 合并
SARdata = gdf.merge(merged_df, on='MSOA_CODE', how='inner')

# 查看结果
print(SARdata.head())

   MSOA_CODE                 geo_label geo_labelw  \
0  E02000730                Newham 017       None   
1  E02000053                Barnet 030       None   
2  E02000243                Ealing 006       None   
3  E02000793  Richmond upon Thames 010       None   
4  E02000077                Bexley 013       None   

                                            geometry  \
0  MULTIPOLYGON (((5.41e+05 1.84e+05, 5.41e+05 1....   
1  MULTIPOLYGON (((5.21e+05 1.9e+05, 5.21e+05 1.9...   
2  MULTIPOLYGON (((5.13e+05 1.84e+05, 5.13e+05 1....   
3  MULTIPOLYGON (((5.13e+05 1.74e+05, 5.13e+05 1....   
4  MULTIPOLYGON (((5.47e+05 1.77e+05, 5.47e+05 1....   

   tourism_economic_indicator  airbnb_supply  Unnamed: 0  hotspot_binary  
0                        20.0           27.0         0.0             0.0  
1                        25.0          201.0         1.0             0.0  
2                         5.0           43.0         2.0             0.0  
3                        20.0           50.0

In [72]:
# Create interaction term: Airbnb supply * traditional tourism dummy
SARdata['airbnb_interact'] = SARdata['airbnb_supply'] * SARdata['hotspot_binary']

In [92]:
# 计算每个区域的面积（m²）
SARdata['area_m2'] = SARdata.geometry.area

# 将面积转换为公顷 (1 ha = 10,000 m²)
SARdata['area_ha'] = SARdata['area_m2'] / 10000.0

# 现在你可以使用 'area_ha' 计算Airbnb密度，例如每公顷房源数
SARdata['airbnb_density'] = SARdata['airbnb_supply'] / SARdata['area_ha']

In [89]:
# 创建交互项
SARdata['airbnb_interact'] = SARdata['airbnb_density'] * SARdata['hotspot_binary']

# 因变量
y = SARdata['tourism_economic_indicator'].values.reshape(-1,1)

# 自变量，根据需求加入控制变量
X_vars = ['airbnb_density', 'airbnb_interact']
# 如果有控制变量，如 population, income，请加入：
# X_vars = ['airbnb_supply', 'airbnb_interact', 'population', 'income']

X = SARdata[X_vars].values

# 构建空间权重矩阵
W = weights.contiguity.Queen.from_dataframe(SARdata, use_index=True)
W.transform = 'r'

# 可选：查看Moran's I 判断空间相关性
mi = esda.moran.Moran(y.flatten(), W)
print("Moran's I:", mi.I, "p-value:", mi.p_sim)

Moran's I: 0.4414711296308822 p-value: 0.001


In [90]:
# 在存在空间自相关情况下，使用SAR模型（以ML_Lag为例）
sar_model = ML_Lag(
    y, 
    X, 
    w=W, 
    name_y='tourism_economic_indicator',
    name_x=X_vars,
    name_w='W',
    method='full'
)

print(sar_model.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :           W
Dependent Variable  :tourism_economic_indicator                Number of Observations:         983
Mean dependent var  :     40.0814                Number of Variables   :           4
S.D. dependent var  :    103.3930                Degrees of Freedom    :         979
Pseudo R-squared    :      0.4769
Spatial Pseudo R-squared:  0.1295
Log likelihood      :  -5725.1483
Sigma-square ML     :   5914.0135                Akaike info criterion :   11458.297
S.E of regression   :     76.9026                Schwarz criterion     :   11477.859

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-------------------------------------------------